# Model Configuration Testing Lab

This notebook tests different approaches to configuring Bedrock models in SageMaker Unified Studio.

## Key Discovery

Bedrock IDE-created profiles have a special tag `AmazonBedrockManaged=true` that CLI-created profiles lack.

## Goals

1. Understand what makes Bedrock IDE profiles work
2. Test if we can replicate the profile creation
3. Document findings for repeatable lab setup

## 1. Install Dependencies

In [ ]:
# Install required packages
%pip install langgraph>=1.0.6 boto3 -q

## 2. Imports and Setup

In [ ]:
import boto3
import json
from datetime import datetime
from typing import Optional

# Initialize clients
bedrock = boto3.client('bedrock', region_name='us-west-2')
sts = boto3.client('sts')

# Get account info
identity = sts.get_caller_identity()
ACCOUNT_ID = identity['Account']
REGION = 'us-west-2'

print(f"Account: {ACCOUNT_ID}")
print(f"Region: {REGION}")
print(f"User ARN: {identity['Arn']}")

## 3. Discover Existing Profiles

List and compare existing inference profiles to understand the differences.

In [ ]:
def list_application_profiles():
    """List all application inference profiles with their details."""
    response = bedrock.list_inference_profiles(typeEquals='APPLICATION')
    profiles = response.get('inferenceProfileSummaries', [])
    
    print(f"Found {len(profiles)} application inference profiles:\n")
    print("=" * 80)
    
    for p in profiles:
        print(f"Name: {p['inferenceProfileName']}")
        print(f"ARN:  {p['inferenceProfileArn']}")
        print(f"Description: {p.get('description', 'N/A')}")
        print(f"Status: {p['status']}")
        print(f"Created: {p['createdAt']}")
        
        # Get tags
        try:
            tags_response = bedrock.list_tags_for_resource(resourceARN=p['inferenceProfileArn'])
            tags = {t['key']: t['value'] for t in tags_response.get('tags', [])}
            print(f"Tags: {json.dumps(tags, indent=2)}")
        except Exception as e:
            print(f"Tags: Error fetching - {e}")
        
        print("-" * 80)
    
    return profiles

profiles = list_application_profiles()

## 4. Analyze Profile Differences

Compare Bedrock IDE-created profiles vs CLI-created profiles.

In [ ]:
def analyze_profile(arn: str) -> dict:
    """Get full profile details including tags."""
    profile = bedrock.get_inference_profile(inferenceProfileIdentifier=arn)
    tags_response = bedrock.list_tags_for_resource(resourceARN=arn)
    tags = {t['key']: t['value'] for t in tags_response.get('tags', [])}
    
    return {
        'name': profile['inferenceProfileName'],
        'arn': profile['inferenceProfileArn'],
        'description': profile.get('description', ''),
        'models': [m['modelArn'] for m in profile.get('models', [])],
        'tags': tags,
        'is_bedrock_managed': tags.get('AmazonBedrockManaged') == 'true',
        'datazone_project': tags.get('AmazonDataZoneProject'),
        'datazone_domain': tags.get('AmazonDataZoneDomain'),
    }

print("Profile Analysis:")
print("=" * 80)

for p in profiles:
    analysis = analyze_profile(p['inferenceProfileArn'])
    print(f"\nProfile: {analysis['name']}")
    print(f"  Bedrock Managed: {'YES' if analysis['is_bedrock_managed'] else 'NO'}")
    print(f"  DataZone Project: {analysis['datazone_project']}")
    print(f"  DataZone Domain: {analysis['datazone_domain']}")
    print(f"  Description pattern: {'SageMaker Studio' if 'SageMaker' in analysis['description'] else 'Custom'}")

## 5. Key Findings

Document the differences between working and non-working profiles.

In [ ]:
print("""
============================================================
KEY FINDINGS: What Makes Bedrock IDE Profiles Work
============================================================

1. NAMING CONVENTION:
   - Bedrock IDE uses: "{domain_id} {project_id}"
   - Example: "dzd-69hyksbfyg5nps 5elox3qmz8kyz4"

2. DESCRIPTION FORMAT:
   - "Created by Amazon SageMaker Unified Studio for domain {domain_id} 
      to provide access to Amazon Bedrock model in project {project_id}"

3. REQUIRED TAGS:
   - AmazonDataZoneProject: {project_id}
   - AmazonDataZoneDomain: {domain_id}
   - AmazonBedrockManaged: true   <-- CRITICAL!

4. MODEL SOURCE:
   - Both use the same cross-region inference profile as source
   - arn:aws:bedrock:{region}:{account}:inference-profile/us.anthropic.claude-*

HYPOTHESIS:
The `AmazonBedrockManaged=true` tag may trigger special IAM handling
in the SageMaker permissions boundary policy.
============================================================
""")

## 6. Test: Create Profile with Bedrock IDE Pattern

Attempt to create a profile that mimics the Bedrock IDE pattern.

In [ ]:
import os
import glob

def detect_datazone_ids():
    """Auto-detect DataZone IDs from Bedrock IDE exports."""
    # Look for export folders
    patterns = [
        '../amazon-bedrock-ide-app-export-*/amazon-bedrock-ide-app-stack-*.json',
        './amazon-bedrock-ide-app-export-*/amazon-bedrock-ide-app-stack-*.json',
        '../../amazon-bedrock-ide-app-export-*/amazon-bedrock-ide-app-stack-*.json',
    ]
    
    for pattern in patterns:
        files = glob.glob(pattern)
        for f in files:
            try:
                with open(f) as fp:
                    content = fp.read()
                    # Extract project ID
                    import re
                    project_match = re.search(r'"exportProjectId":\s*"([^"]+)"', content)
                    domain_match = re.search(r'(dzd-[a-z0-9]+)', content)
                    
                    if project_match and domain_match:
                        return {
                            'project_id': project_match.group(1),
                            'domain_id': domain_match.group(1),
                            'source_file': f
                        }
            except Exception as e:
                continue
    
    return None

datazone = detect_datazone_ids()
if datazone:
    print(f"Detected DataZone IDs from: {datazone['source_file']}")
    print(f"  Project ID: {datazone['project_id']}")
    print(f"  Domain ID:  {datazone['domain_id']}")
    
    DATAZONE_PROJECT_ID = datazone['project_id']
    DATAZONE_DOMAIN_ID = datazone['domain_id']
else:
    print("Could not auto-detect DataZone IDs.")
    print("Please set manually:")
    DATAZONE_PROJECT_ID = "YOUR_PROJECT_ID"
    DATAZONE_DOMAIN_ID = "dzd-YOUR_DOMAIN_ID"

In [ ]:
def create_bedrock_managed_profile(
    project_id: str,
    domain_id: str,
    model_key: str = 'sonnet'
) -> dict:
    """
    Create an inference profile mimicking Bedrock IDE pattern.
    
    Key elements:
    1. Name format: "{domain_id} {project_id}"
    2. Description mentioning SageMaker Unified Studio
    3. Tags including AmazonBedrockManaged=true
    """
    
    # Model mapping
    models = {
        'haiku': 'us.anthropic.claude-3-5-haiku-20241022-v1:0',
        'sonnet': 'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
        'sonnet4': 'us.anthropic.claude-sonnet-4-20250514-v1:0',
    }
    
    model_id = models.get(model_key, models['sonnet'])
    model_source_arn = f"arn:aws:bedrock:{REGION}:{ACCOUNT_ID}:inference-profile/{model_id}"
    
    # Bedrock IDE naming pattern
    profile_name = f"{domain_id} {project_id} lab"
    description = f"Created by lab script for domain {domain_id} to provide access to Amazon Bedrock model in project {project_id}"
    
    print(f"Creating profile with Bedrock IDE pattern:")
    print(f"  Name: {profile_name}")
    print(f"  Description: {description}")
    print(f"  Model Source: {model_source_arn}")
    print()
    
    # Tags - including the critical AmazonBedrockManaged tag
    tags = [
        {'key': 'AmazonDataZoneProject', 'value': project_id},
        {'key': 'AmazonDataZoneDomain', 'value': domain_id},
        {'key': 'AmazonBedrockManaged', 'value': 'true'},  # THE KEY TAG!
        {'key': 'Purpose', 'value': 'LangGraphLab'},
    ]
    
    print(f"  Tags: {json.dumps(tags, indent=4)}")
    print()
    
    try:
        response = bedrock.create_inference_profile(
            inferenceProfileName=profile_name,
            description=description,
            modelSource={'copyFrom': model_source_arn},
            tags=tags
        )
        
        print("SUCCESS! Profile created:")
        print(f"  ARN: {response['inferenceProfileArn']}")
        print(f"  Status: {response['status']}")
        return response
        
    except bedrock.exceptions.ConflictException:
        print("Profile already exists. Looking up existing profile...")
        profiles = bedrock.list_inference_profiles(typeEquals='APPLICATION')
        for p in profiles.get('inferenceProfileSummaries', []):
            if p['inferenceProfileName'] == profile_name:
                print(f"  Existing ARN: {p['inferenceProfileArn']}")
                return {'inferenceProfileArn': p['inferenceProfileArn'], 'status': p['status']}
    except Exception as e:
        print(f"ERROR: {e}")
        return None

# Only run if we have DataZone IDs
if 'DATAZONE_PROJECT_ID' in dir() and not DATAZONE_PROJECT_ID.startswith('YOUR'):
    result = create_bedrock_managed_profile(DATAZONE_PROJECT_ID, DATAZONE_DOMAIN_ID)
else:
    print("Skipping - DataZone IDs not configured")

## 7. Test: Invoke LLM with Different Profiles

Test each profile type to see which works.

In [ ]:
from langchain_aws import ChatBedrockConverse\nfrom langchain_core.messages import HumanMessage\n\ndef test_profile(profile_arn: str, description: str) -> bool:\n    \"\"\"\n    Test if a profile can be used to invoke the model.\n    Returns True if successful, False otherwise.\n    \"\"\"\n    print(f\"\\nTesting: {description}\")\n    print(f\"ARN: {profile_arn}\")\n    print(\"-\" * 60)\n    \n    try:\n        llm = ChatBedrockConverse(\n            model=profile_arn,\n            provider=\"anthropic\",\n            region_name=REGION,\n            temperature=0,\n        )\n        \n        response = llm.invoke([HumanMessage(content=\"Say 'Hello from Bedrock' in exactly 5 words.\")])\n        print(f\"SUCCESS! Response: {response.content}\")\n        return True\n        \n    except Exception as e:\n        error_msg = str(e)\n        if 'AccessDeniedException' in error_msg:\n            print(f\"FAILED: Access Denied\")\n            # Extract the specific error\n            if 'not authorized' in error_msg.lower():\n                print(f\"  Reason: IAM permissions boundary blocked the request\")\n        else:\n            print(f\"FAILED: {error_msg[:200]}\")\n        return False\n\n# Collect all profiles to test\ntest_cases = []\n\n# Add manually created test profile with AmazonBedrockManaged=true tag\ntest_cases.append({\n    'arn': 'arn:aws:bedrock:us-west-2:159878781974:application-inference-profile/w120tuwxb2o8',\n    'description': 'CLI-created with AmazonBedrockManaged=true (TEST)'\n})\n\n# Get existing profiles\nfor p in profiles:\n    analysis = analyze_profile(p['inferenceProfileArn'])\n    test_cases.append({\n        'arn': p['inferenceProfileArn'],\n        'description': f\"{analysis['name']} (BedrockManaged={analysis['is_bedrock_managed']})\"\n    })\n\nprint(\"=\" * 60)\nprint(\"PROFILE TESTING RESULTS\")\nprint(\"=\" * 60)\n\nresults = []\nfor tc in test_cases:\n    success = test_profile(tc['arn'], tc['description'])\n    results.append({'profile': tc['description'], 'success': success})\n\nprint(\"\\n\" + \"=\" * 60)\nprint(\"SUMMARY\")\nprint(\"=\" * 60)\nfor r in results:\n    status = \"WORKS\" if r['success'] else \"BLOCKED\"\n    print(f\"  [{status}] {r['profile']}\")

## 8. Test Direct Model IDs (Expected to Fail)

Confirm that direct model IDs don't work in SageMaker Unified Studio.

In [ ]:
direct_models = [
    ('anthropic.claude-3-5-sonnet-20241022-v2:0', 'Base model ID'),
    ('us.anthropic.claude-3-5-sonnet-20241022-v2:0', 'Cross-region model ID'),
]

print("=" * 60)
print("DIRECT MODEL ID TESTING (expected to fail in SageMaker Studio)")
print("=" * 60)

for model_id, description in direct_models:
    print(f"\nTesting: {description}")
    print(f"Model: {model_id}")
    print("-" * 60)
    
    try:
        llm = ChatBedrockConverse(
            model=model_id,
            region_name=REGION,
            temperature=0,
        )
        
        response = llm.invoke([HumanMessage(content="Say 'test' once.")])
        print(f"SUCCESS! (Unexpected in SageMaker Studio)")
        print(f"Response: {response.content}")
        
    except Exception as e:
        error_msg = str(e)
        if 'AccessDeniedException' in error_msg:
            print(f"FAILED: Access Denied (expected in SageMaker Studio)")
        else:
            print(f"FAILED: {error_msg[:200]}")

## 9. Conclusions and Recommendations

In [ ]:
print("""
============================================================
CONCLUSIONS
============================================================

1. WHAT WORKS:
   - Application inference profiles created by Bedrock IDE
   - Must have `AmazonBedrockManaged=true` tag
   - Must have correct DataZone project/domain tags
   - Must use `provider="anthropic"` in ChatBedrockConverse

2. WHAT DOESN'T WORK:
   - Direct model IDs (anthropic.claude-*)
   - Cross-region model IDs (us.anthropic.claude-*)
   - CLI-created profiles WITHOUT `AmazonBedrockManaged=true` tag

3. HYPOTHESIS:
   The SageMaker permissions boundary policy likely has a condition like:
   
   "Condition": {
     "StringEquals": {
       "aws:ResourceTag/AmazonBedrockManaged": "true"
     }
   }
   
   This means only resources tagged as "managed by Bedrock" are allowed.

4. SOLUTION OPTIONS:
   
   Option A: Use Bedrock IDE (Recommended for production)
   - Create app in Bedrock IDE UI
   - Export and extract the profile ARN
   - Most reliable, officially supported
   
   Option B: CLI with AmazonBedrockManaged tag (Testing)
   - May work if the tag is the only missing piece
   - Not officially supported, may break
   
   Option C: Run outside SageMaker Unified Studio
   - Use SageMaker Classic, EC2, or local
   - Direct model IDs work in unrestricted environments

============================================================
""")

# Generate recommended configuration
working_profile = None
for p in profiles:
    analysis = analyze_profile(p['inferenceProfileArn'])
    if analysis['is_bedrock_managed']:
        working_profile = p['inferenceProfileArn']
        break

if working_profile:
    print(f"RECOMMENDED CONFIGURATION:")
    print(f"")
    print(f'INFERENCE_PROFILE_ARN = "{working_profile}"')
    print(f'REGION = "{REGION}"')
    print(f"")
    print(f"llm = ChatBedrockConverse(")
    print(f"    model=INFERENCE_PROFILE_ARN,")
    print(f'    provider="anthropic",  # Required for ARN')
    print(f"    region_name=REGION,")
    print(f"    temperature=0,")
    print(f")")

## 10. Utility: Find Working Profile Automatically

In [ ]:
def find_working_profile(region: str = 'us-west-2') -> Optional[str]:
    """
    Find a working application inference profile.
    Prioritizes profiles with AmazonBedrockManaged=true tag.
    
    Returns the profile ARN or None if not found.
    """
    client = boto3.client('bedrock', region_name=region)
    
    response = client.list_inference_profiles(typeEquals='APPLICATION')
    profiles = response.get('inferenceProfileSummaries', [])
    
    # First pass: find profiles with AmazonBedrockManaged=true
    for p in profiles:
        try:
            tags_response = client.list_tags_for_resource(resourceARN=p['inferenceProfileArn'])
            tags = {t['key']: t['value'] for t in tags_response.get('tags', [])}
            
            if tags.get('AmazonBedrockManaged') == 'true':
                print(f"Found Bedrock-managed profile: {p['inferenceProfileName']}")
                return p['inferenceProfileArn']
        except:
            continue
    
    # Second pass: return any application profile as fallback
    if profiles:
        print(f"Warning: No Bedrock-managed profile found. Using: {profiles[0]['inferenceProfileName']}")
        return profiles[0]['inferenceProfileArn']
    
    return None

# Test the utility
working_arn = find_working_profile()
if working_arn:
    print(f"\nUse this ARN in your notebooks:")
    print(f'INFERENCE_PROFILE_ARN = "{working_arn}"')
else:
    print("No application inference profiles found.")
    print("Create one in Bedrock IDE first.")

## Next Steps

1. **If `AmazonBedrockManaged=true` tag works**: Update `setup-inference-profile.sh` to include this tag
2. **If it doesn't work**: Document that Bedrock IDE is required and provide automation for extracting the ARN
3. **For labs**: Consider providing pre-created profiles or CloudFormation templates